In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras import backend as K

# Load MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()


In [ ]:
# Normalize and flatten images
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Deterministic Autoencoder
input_dim = x_train.shape[1]
latent_dim = 2  # 2-dimensional latent space

# Encoder
input_img = Input(shape=(input_dim,))
encoded = Dense(latent_dim, activation='relu')(input_img)

# Decoder
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Define autoencoder model
autoencoder = Model(input_img, decoded)

# Compile and train the autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=10, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

# Extract the encoder model
encoder = Model(input_img, encoded)

# Generate latent space points using deterministic autoencoder
latent_points_deterministic = encoder.predict(x_test)

# Variational Autoencoder (VAE)
# Reparameterization trick
def sampling(args):
z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Encoder
input_img = Input(shape=(input_dim,))
z_mean = Dense(latent_dim)(input_img)
z_log_var = Dense(latent_dim)(input_img)
z = Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoded = Dense(input_dim, activation='sigmoid')(z)

# Define VAE model
vae = Model(input_img, decoded)

# Compute VAE loss
reconstruction_loss = keras.losses.binary_crossentropy(input_img, decoded)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)

# Compile and train the VAE
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.fit(x_train, epochs=10, batch_size=256, shuffle=True, validation_data=(x_test, None))

# Extract the encoder model
encoder = Model(input_img, z_mean)

# Generate latent space points using VAE
latent_points_vae = encoder.predict(x_test)

# Visualize the latent space
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.scatter(latent_points_deterministic[:, 0], latent_points_deterministic[:, 1], c='b', cmap='rainbow')
plt.title('Deterministic Autoencoder')

plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.colorbar()


In [ ]:
plt.subplot(1, 2, 2)
plt.scatter(latent_points_vae[:, 0], latent_points_vae[:, 1], c='b', cmap='rainbow')
plt.title('Variational Autoencoder (VAE)')

plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.colorbar()

plt.tight_layout()
plt.show()
